# Final Report

* Author : 楊右宇
* Establish Date：2021.12.24
* Last Modified Date：2021.01.03

# import library

In [3]:
!pip install mplfinance

     |████████████████████████████████| 64 kB 769 kB/s            


In [4]:
import pandas as pd
import numpy as np
import mplfinance as mpf
import math      ### 410974045 ### math.floor取無條件捨去值 ###

# Define Control Variables

In [12]:
DATA_SOURCE = r"../data/0050.csv"      ### 方便定義 data source ###

# Indicators

## 當日漲跌幅

In [6]:
def variation(_df):

    res = _df.copy(deep=True)
    
    res['variation'] = np.nan

    col_var = res.columns.get_loc('variation')
    col_close = _df.columns.get_loc('Close')
    
    for i in range(1, len(_df)):
        res.iloc[i, col_var] = ((_df.iloc[i, col_close] - _df.iloc[i - 1, col_close]) / _df.iloc[i - 1, col_close]) * 100

    return res

## Moving Average

In [7]:
def MA(_df, n_days):
    """ make the new colume names as <ma_@n_days>, the value 
        will be the @n_days average of the @Close price"""

    res = _df.copy(deep=True)
    res['ma_' + str(n_days)] = _df['Close'].rolling(n_days).mean()
    return res

## RSI

In [8]:
def RSI(_ticker, n_days = 14):
    
    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=_ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    res = _ticker.copy(deep=True)

    tmp['rise'] = np.nan

    col_rise = tmp.columns.get_loc('rise')
    col_close = _ticker.columns.get_loc('Close')

    for i in range(1, len(_ticker)):
        tmp.iloc[i, col_rise] = (_ticker.iloc[i, col_close] - _ticker.iloc[i - 1, col_close])


    tmp['rise_only'] = tmp['rise']
    tmp['drop_only'] = tmp['rise']

    tmp['rise_only'].loc[tmp['rise'] < 0] = 0
    tmp['drop_only'].loc[tmp['rise'] > 0] = 0
    
    tmp['rise_avg'] = tmp['rise_only'].rolling(n_days).mean()
    tmp['drop_avg'] = tmp['drop_only'].rolling(n_days).mean().abs()

    res['RSI'] = tmp['rise_avg'] / (tmp['rise_avg'] + tmp['drop_avg']) * 100
    
    return res

## KD

In [9]:
def KD(_ticker, n_days = 9):

    # Create a empty dataFrame with original index
    # To store some temporary data
    tmp = pd.DataFrame(index=_ticker.index)

    # Copy a dataFrame to store result
    # deep=True means make a copy instead of link
    res = _ticker.copy(deep=True)

    tmp['max_close'] = ticker['Close'].rolling(n_days).max()
    tmp['min_close'] = ticker['Close'].rolling(n_days).min()
    # tmp['max_close'] = ticker['Close'].rolling(n_days, min_periods=1).max()
    # tmp['min_close'] = ticker['Close'].rolling(n_days, min_periods=1).min()

    # RSV also called FastK
    tmp['RSV'] = (ticker['Close'] - tmp['min_close']) / (tmp['max_close'] - tmp['min_close']) * 100

    res['K'] = .0
    res['D'] = .0
    
    col_k = res.columns.get_loc('K')
    col_d = res.columns.get_loc('D')
    col_rsv = tmp.columns.get_loc('RSV')

    # for i in range(1, len(_ticker)):
    for i in range(9, len(_ticker)):
        res.iloc[i, col_k] = res.iloc[i - 1, col_k] * 2 / 3 + tmp.iloc[i, col_rsv] * 1 / 3
        res.iloc[i, col_d] = res.iloc[i - 1, col_d] * 2 / 3 + res.iloc[i, col_k] * 1 / 3
    
    return res

# 資料處裡

In [20]:
# 檔案無欄位名稱時，加 -> names =  ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
# df = pd.read_csv(r"C:\Coding\43_School_S3\investment\week14_RSI_KD\RSI\0050.csv", index_col=0, parse_dates=True)

ticker = pd.read_csv(DATA_SOURCE)

# change the name of columns
ticker.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# set the column "Date" to index
ticker = ticker.set_index('Date')

# set index column("Date") from string to datetime.datetime
ticker.index = pd.to_datetime(ticker.index)

# sort index column("Date") chronologically
ticker = ticker.sort_index()


ticker = MA(ticker, 5)
ticker = MA(ticker, 10)
ticker = MA(ticker, 20)
# print(ticker.head(50))
ticker = variation(ticker)
ticker = RSI(ticker)
ticker = KD(ticker)


ticker = ticker.loc['2021-01':'2021-10', ]
print(ticker)

              Open    High     Low   Close  Volume    ma_5    ma_10     ma_20  \
Date                                                                            
2021-01-04  122.20  124.50  122.00  124.35    6307  121.62  120.135  119.4275   
2021-01-05  124.20  124.65  123.75  124.60    4962  122.54  120.660  119.7375   
2021-01-06  126.10  127.15  124.10  125.95   10859  123.75  121.455  120.0325   
2021-01-07  126.55  129.00  126.55  128.90    6863  125.21  122.530  120.4750   
2021-01-08  130.60  131.25  129.55  131.20   13298  127.00  123.770  121.1050   
2021-01-11  131.20  132.30  130.50  132.30   10492  128.59  125.105  121.7825   
2021-01-12  132.30  133.15  131.60  132.10   11223  130.09  126.315  122.4750   
2021-01-13  132.75  135.45  132.70  135.30   10912  131.96  127.855  123.3875   
2021-01-14  135.00  135.05  133.35  134.05   12390  132.99  129.100  124.1275   
2021-01-15  137.15  138.00  134.40  134.50   21933  133.65  130.325  124.9225   
2021-01-18  134.60  134.95  

# 交易策略

## RSI 買賣超

In [14]:
def bound(_ticker):
    
    upperbound = 95
    lowerbound = 15
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')

    for i in range(len(_ticker)):
        if(_ticker['RSI'][i] < lowerbound):
            res.iloc[i, col_b] = 1

        elif(_ticker['RSI'][i] > upperbound):
            res.iloc[i, col_b] = -1
            
    return res

## 短中長均線呈現多方趨勢

In [15]:
def ma_strategy(_ticker):
    
    ## 如果找不到指標要在此處加指標

    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i], 'ma_5'] > _ticker.loc[_ticker.index[i], "ma_10"] > _ticker.loc[_ticker.index[i], "ma_20"] \
            and (_ticker.loc[_ticker.index[i], 'ma_5'] - _ticker.loc[_ticker.index[i - 1], 'ma_5']) \
                / _ticker.loc[_ticker.index[i - 1], 'ma_5'] > 0.01 \
            and (_ticker.loc[_ticker.index[i], "ma_20"] - _ticker.loc[_ticker.index[i - 1], "ma_20"]) \
                / _ticker.loc[_ticker.index[i - 1], "ma_20"] > 0.007 :

            res.iloc[i, col_b] = 1

        elif _ticker.loc[_ticker.index[i], 'Close'] < _ticker.loc[_ticker.index[i], 'ma_20']:
            res.iloc[i, col_b] = -1
            
    return res

## KD Cross in Oversold / Overbought Regions

In [16]:
### 410974045 ###
def kd_strategy(_ticker):
    ## 當KD值位於20下方向上做黃金交叉時買進，反之處於80上方向下做死亡交叉時賣出
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i-1], 'K'] < 20 and _ticker.loc[_ticker.index[i-1], 'D'] < 20 \
           and _ticker.loc[_ticker.index[i], 'K'] > _ticker.loc[_ticker.index[i], 'D'] \
           and _ticker.loc[_ticker.index[i-1], 'K'] < _ticker.loc[_ticker.index[i-1], 'D'] :
            res.iloc[i, col_b] = 1
        elif _ticker.loc[_ticker.index[i-1], 'K'] > 80 and _ticker.loc[_ticker.index[i-1], 'D'] > 80 \
           and _ticker.loc[_ticker.index[i], 'K'] < _ticker.loc[_ticker.index[i], 'D'] \
           and _ticker.loc[_ticker.index[i-1], 'K'] > _ticker.loc[_ticker.index[i-1], 'D'] :
            res.iloc[i, col_b] = -1
            
    return res

## RSI Indexing While KD Cross

In [17]:
### 410974045 ###
def rsi_kd_strategy(_ticker):
    ## 當RSI>50且KD值向上形成黃金交叉時買進；反之RSI>80且KD向下形成死亡交叉時賣出
    
    res = _ticker.copy(deep=True)
    res['buy_point'] = 0
    col_b = res.columns.get_loc('buy_point')

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i-1], 'RSI'] > 50 \
           and _ticker.loc[_ticker.index[i], 'K'] > _ticker.loc[_ticker.index[i], 'D'] \
           and _ticker.loc[_ticker.index[i-1], 'K'] < _ticker.loc[_ticker.index[i-1], 'D'] :
            res.iloc[i, col_b] = 1
        elif _ticker.loc[_ticker.index[i-1], 'RSI'] > 80 \
           and _ticker.loc[_ticker.index[i], 'K'] < _ticker.loc[_ticker.index[i], 'D'] \
           and _ticker.loc[_ticker.index[i-1], 'K'] > _ticker.loc[_ticker.index[i-1], 'D'] :
            res.iloc[i, col_b] = -1
            
    return res

# 資料回測

In [18]:
def print_info(_ticker):

    funds = 1_000_000
    buys_num = 0
    res = pd.DataFrame(index=_ticker.index)

    res['ticker_num'] = 0
    res['income'] = 0
    res['expenditure'] = 0
    res['account'] = funds

    for i in range(1, len(_ticker)):
        if _ticker.loc[_ticker.index[i - 1], 'buy_point'] == 1 and res.loc[_ticker.index[i - 1], 'ticker_num'] == 0 :
            res.loc[_ticker.index[i], 'ticker_num'] \
                = math.floor(res.loc[_ticker.index[i - 1], 'account'] / (_ticker.loc[_ticker.index[i], 'Open'] * 1.001425 * 1000))
                ### 410974045 ### 最小取整 ###
                # = round(res.loc[_ticker.index[i - 1], 'account'] / (_ticker.loc[_ticker.index[i], 'Open'] * 1.001425 * 1000))
            res.loc[_ticker.index[i], 'expenditure'] \
                = -res.loc[_ticker.index[i], 'ticker_num'] * (_ticker.loc[_ticker.index[i], 'Open'] * 1.001425 * 1000)
            res.loc[_ticker.index[i], 'account'] = res.loc[_ticker.index[i - 1], 'account'] + res.loc[_ticker.index[i], 'expenditure']
            
        elif _ticker.loc[_ticker.index[i - 1], 'buy_point'] == -1 and res.loc[_ticker.index[i - 1], 'ticker_num'] > 0 :
            print(_ticker.index[i])
            res.loc[_ticker.index[i], 'income'] \
                = res.loc[_ticker.index[i - 1], 'ticker_num'] * (_ticker.loc[_ticker.index[i], 'Open'] * 0.995575 * 1000)   # 0.995575 = 1-(0.003(交易稅)+0.001425)
            res.loc[_ticker.index[i], 'ticker_num'] = 0
            res.loc[_ticker.index[i], 'account'] = res.loc[_ticker.index[i - 1], 'account'] + res.loc[_ticker.index[i], 'income']

        elif i == len(_ticker) - 1 and res.loc[_ticker.index[i - 1], 'ticker_num'] > 0 :
            res.loc[_ticker.index[i], 'income'] \
                = res.loc[_ticker.index[i - 1], 'ticker_num'] * (_ticker.loc[_ticker.index[i], 'Open'] * 0.995575 * 1000)
            res.loc[_ticker.index[i], 'ticker_num'] = 0
            res.loc[_ticker.index[i], 'account'] = res.loc[_ticker.index[i - 1], 'account'] + res.loc[_ticker.index[i], 'income']
        else :
            res.loc[_ticker.index[i], 'ticker_num'] = res.loc[_ticker.index[i - 1], 'ticker_num']
            res.loc[_ticker.index[i], 'account'] = res.loc[_ticker.index[i - 1], 'account']

    return res

# 實作

In [19]:
### r = ma_strategy(ticker)
### 410974045 ###
# r = kd_strategy(ticker)
r = rsi_kd_strategy(ticker)
### 測試結果
r.to_csv('0050_with_indicator.csv')
print(r.head(50))
info = print_info(r)
info.to_csv('Simulation_Result.csv')
print(info.head(50))
print(info.tail(50))

             Open   High    Low  Close  Volume    ma_5   ma_10    ma_20  \
Date                                                                      
2003-06-30  37.10  37.40  36.92  37.08    9930     NaN     NaN      NaN   
2003-07-01  37.09  38.10  37.09  38.05   14290     NaN     NaN      NaN   
2003-07-02  38.17  38.82  38.10  38.69   16232     NaN     NaN      NaN   
2003-07-03  40.60  40.60  38.81  39.00   16839     NaN     NaN      NaN   
2003-07-04  39.10  39.26  38.75  39.26   12491  38.416     NaN      NaN   
2003-07-07  39.60  41.00  39.60  41.00   19461  39.200     NaN      NaN   
2003-07-08  42.00  42.00  40.70  41.19   13945  39.828     NaN      NaN   
2003-07-09  41.30  41.30  40.81  41.22    8849  40.334     NaN      NaN   
2003-07-10  41.20  41.20  40.05  40.05    8603  40.544     NaN      NaN   
2003-07-11  39.60  39.97  39.30  39.91    7019  40.674  39.545      NaN   
2003-07-14  39.95  41.00  39.92  40.66   10771  40.606  39.903      NaN   
2003-07-15  40.70  40.86 